## Google Earth Engine (GEE) JavaScript API — Exercise
Using Sentinel2 - Level 2A, calculate the NDVI index averaged by year from 2018 to 2024 of a study area and export the image as average on the entire period.

### Define your study area and dates

In [ ]:
var studyArea = ee.Geometry.Polygon(
  [[[11.0, 45.0],
    [11.0, 44.5],
    [11.7, 44.5],
    [11.7, 45.0]]]
);

var startYear = 2018;
var endYear   = 2024;

### Collect Sentinel 2 images

In [ ]:
var s2collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  .filterBounds(studyArea)
  .filterDate(ee.Date.fromYMD(startYear, 1, 1) ,ee.Date.fromYMD(endYear, 12, 31))
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20));


### Cloud mask for Sentinel-2 Level 1C using QA60 band

In [ ]:
function maskS2clouds(image){
  var nodata = image.select('SCL').eq([0]);
  var saturated = image.select('SCL').eq([1]);
  var dark_area = image.select('SCL').eq([2]);
  var cloud_shadow = image.select('SCL').eq([3]);
  var cloud_low = image.select('SCL').eq([7]);
  var cloud_med = image.select('SCL').eq([8]);
  var cloud_high = image.select('SCL').eq([9]); 
  var cloud_cirrus = image.select('SCL').eq([10]);
  var cloud_mask = cloud_shadow.add(cloud_low).add(cloud_med).add(cloud_high).add(cloud_cirrus).add(nodata).add(saturated).add(dark_area); 
  var invert_mask = cloud_mask.eq(0).selfMask();
  return image.updateMask(invert_mask);
}

var masking_water = function(image){
  var water = image.select('SCL').eq([6]);
  var snow = image.select('SCL').eq([11]);
  var mask = water.add(snow); 
  var invert_mask = mask.eq(0).selfMask();
  return image.updateMask(invert_mask);
}


### Add NDVI band: (NIR - RED) / (NIR + RED)

In [ ]:
function addNDVI(image) {
  var ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI');
  return image.addBands(ndvi);
}

### Apply cloud mask

In [ ]:
var s2=s2collection.map(maskS2clouds).map(masking_water)
  .map(addNDVI);

### Build a FeatureCollection of yearly mean NDVI

In [ ]:
var years = ee.List.sequence(startYear, endYear);

var yearlyNdviFc = ee.ImageCollection(
  years.map(function(y) {
    y = ee.Number(y);
    var start = ee.Date.fromYMD(y, 1, 1);
    var end   = ee.Date.fromYMD(y, 12, 31);

    // NDVI images for that year
    var ndviYear = s2
      .filterDate(start, end)
      .select('NDVI');

    // Mean NDVI image over that year
    var mean_ndviYear = ndviYear.mean();
    return mean_ndviYear.set('system:time_start',y);
  })
);

### Plot the NDVI time-series of a point

In [ ]:
var randomPointFc = ee.FeatureCollection.randomPoints({
    region: studyArea,
    points: 1,
    seed: 42
  });
  
  var randomPoint = ee.Feature(randomPointFc.first()).geometry();

In [ ]:
var ndviTsChart = ui.Chart.image.series({
    imageCollection: yearlyNdviFc.select('NDVI'),
    region: randomPoint,
    reducer: ee.Reducer.mean(),
    scale: 10
  })
  .setOptions({
    title: 'NDVI time series at random point',
    hAxis: {title: 'Date'},
    vAxis: {title: 'NDVI'},
    lineWidth: 2,
    pointSize: 3
  });
  
  print('NDVI trend at random point', ndviTsChart);

### Calculate the mean NDVI in the period

In [ ]:
var medianNdvi = s2.select('NDVI').mean().clip(studyArea);

### Add image to map

In [ ]:
var ndviVis = {
  min: 0,
  max: 1,
  palette: ['#d73027', '#f46d43', '#fee08b', '#d9ef8b', '#66bd63', '#1a9850']
};

Map.centerObject(studyArea, 9);
Map.addLayer(medianNdvi, ndviVis, 'Median NDVI 2016–2024');
Map.addLayer(studyArea, {color: 'red'}, 'Study Area', false);

### Export NDVI image to Google Drive

In [ ]:
Export.image.toDrive({
  image: medianNdvi,
  description: 'Median_NDVI_S2_L1C_' + startYear + '_' + endYear,
  folder: 'GEE_exports',
  fileNamePrefix: 'median_ndvi_s2_l1c_' + startYear + '_' + endYear,
  region: studyArea,
  scale: 10,
  maxPixels: 1e13
});

### Calculate the regional mean and standard deviation in the study area of NDVI mean

In [ ]:
var stats = medianNdvi.reduceRegion({
    reducer: ee.Reducer.mean()
      .combine({
        reducer2: ee.Reducer.stdDev(),
        sharedInputs: true
      }),
    geometry: studyArea,
    scale: 10,       // choose a scale appropriate to your data
    maxPixels: 1e9
  });

## Homework
Calculate the difference of NDVI before and after the extreme rainfall event on 2023-05-16 in central Italy which triggered 80000 landslides. You can calculate the NDVI mean considering 3 month before and after the event. In the folder you can find the study area in shapefile format.
(Advanced: calibrate a NDVI threshold detection with the landslide inventory available at https://doi.org/10.5194/essd-17-1055-2025 to fit the triggered landslides)

## Examples

- [Dynamic Landslide Susceptibility](https://mahnoorahmed5593.users.earthengine.app/view/dynamicsusceptibility)
- [Susceptibility Tool for GEE](https://giacomotitti.users.earthengine.app/view/stgee)
- [Spatial Reduction Tool](https://code.earthengine.google.com/93ccd516ed073447ef85cfea3dcfdeab)